In [ ]:
#
# hw4pr3titanic:  titanic-passenger clasification via nearest neighbors
#

In [ ]:
#
# SUGGESTION:  
# 
# +++ copy-paste-and-alter from the iris- + births + digits-cleaning notebooks into here +++
#
# Here, there can be value in weighting columns with different coefficients... explore!
#

In [207]:
# libraries!
import numpy as np      # numpy is Python's "array" library
import pandas as pd     # Pandas is Python's "data" library ("dataframe" == spreadsheet)

In [256]:
# let's read in our flower data...
# 
# for read_csv, use header=0 when row 0 is a header row
# 
cleaned_filename = "titanic_cleaned.csv"
df_tidy = pd.read_csv(cleaned_filename)   # encoding="utf-8" et al.
print(f"{cleaned_filename} : file read into a pandas dataframe.")
df_tidy

titanic_cleaned.csv : file read into a pandas dataframe.


,pclass,sex,age,sibsp,parch,survived
0,1,1,29.0000,0,0,1
1,1,0,0.9167,1,2,1
2,1,1,2.0000,1,2,0
3,1,0,30.0000,1,2,0
4,1,1,25.0000,1,2,0
...,...,...,...,...,...,...
1259,3,0,45.5000,0,0,0
1262,3,1,14.5000,1,0,0
1264,3,0,26.5000,0,0,0
1265,3,0,27.0000,0,0,0


In [257]:
df_tidy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1004 entries, 0 to 1266
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1004 non-null   int64  
 1   sex       1004 non-null   int64  
 2   age       1004 non-null   float64
 3   sibsp     1004 non-null   int64  
 4   parch     1004 non-null   int64  
 5   survived  1004 non-null   int64  
dtypes: float64(1), int64(5)
memory usage: 54.9 KB


In [258]:
# All of the columns need to be numeric, we'll drop irisname
ROW = 0
COLUMN = 1
df_model1 = df_tidy
df_model1

,pclass,sex,age,sibsp,parch,survived
0,1,1,29.0000,0,0,1
1,1,0,0.9167,1,2,1
2,1,1,2.0000,1,2,0
3,1,0,30.0000,1,2,0
4,1,1,25.0000,1,2,0
...,...,...,...,...,...,...
1259,3,0,45.5000,0,0,0
1262,3,1,14.5000,1,0,0
1264,3,0,26.5000,0,0,0
1265,3,0,27.0000,0,0,0


In [259]:
COLUMNS = df_model1.columns            # "list" of columns
print(f"COLUMNS is {COLUMNS}\n")  
  # It's a "pandas" list, called an Index
  # use it just as a Python list of strings:
print(f"COLUMNS[0] is {COLUMNS[0]}\n")

# let's create a dictionary to look up any column index by name
COL_INDEX = {}
for i, name in enumerate(COLUMNS):
    COL_INDEX[name] = i  # using the name (as key), look up the value (i)
print(f"COL_INDEX is {COL_INDEX}\n\n")

COLUMNS is Index(['pclass', 'sex', 'age', 'sibsp', 'parch', 'survived'], dtype='object')

COLUMNS[0] is pclass

COL_INDEX is {'pclass': 0, 'sex': 1, 'age': 2, 'sibsp': 3, 'parch': 4, 'survived': 5}




In [260]:
S = ['female', 'male']   # int to str
S_INDEX = {'female' : 0, 'male' : 1 }  # str to int

def convert_s(sex):
    """ return the sex index (a unique integer/category) """
    #print(f"converting {speciesname}...")
    return S_INDEX[sex]

# Let's try it out...
for sex in S:
    print(f"{sex} maps to {convert_s(sex)}")

female maps to 0
male maps to 1


In [261]:
# all of scikit-learn's ML routines need numbers, not strings
#   ... even for categories/classifications (like species!)
#   so, we will convert the flower-species to numbers

OUTCOME = ['perished', 'survived']   # int to str
OUTCOME_INDEX = {'perished':0, 'survived' :1}  # str to int

def convert_outcome(outcome):
    """ return the outcome index (1 - survived, 0 - perished) """
    #print(f"converting {speciesname}...")
    return OUTCOME_INDEX[outcome]

# Let's try it out...
for state in OUTCOME:
    print(f"{state} maps to {convert_outcome(state)}")

perished maps to 0
survived maps to 1


In [262]:
# reweighting the classes
df_model2 = df_model1.copy()
df_model2['pclass'][df_model2['pclass'] == 3] *= 3
df_model2['pclass'][df_model2['pclass'] == 2] *= 2
df_model2

/var/folders/yk/v4q7xgf968jgjzrckxn2cqx40000gn/T/ipykernel_1500/3604656441.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model2['pclass'][df_model2['pclass'] == 3] *= 3
/var/folders/yk/v4q7xgf968jgjzrckxn2cqx40000gn/T/ipykernel_1500/3604656441.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model2['pclass'][df_model2['pclass'] == 2] *= 2


,pclass,sex,age,sibsp,parch,survived
0,1,1,29.0000,0,0,1
1,1,0,0.9167,1,2,1
2,1,1,2.0000,1,2,0
3,1,0,30.0000,1,2,0
4,1,1,25.0000,1,2,0
...,...,...,...,...,...,...
1259,9,0,45.5000,0,0,0
1262,9,1,14.5000,1,0,0
1264,9,0,26.5000,0,0,0
1265,9,0,27.0000,0,0,0


In [263]:
# reweighting the sex
df_model3 = df_model2.copy()
df_model3['sex'] *= 5
df_model3

,pclass,sex,age,sibsp,parch,survived
0,1,5,29.0000,0,0,1
1,1,0,0.9167,1,2,1
2,1,5,2.0000,1,2,0
3,1,0,30.0000,1,2,0
4,1,5,25.0000,1,2,0
...,...,...,...,...,...,...
1259,9,0,45.5000,0,0,0
1262,9,5,14.5000,1,0,0
1264,9,0,26.5000,0,0,0
1265,9,0,27.0000,0,0,0


In [266]:
# reweighting mothers with children or children with parents
df_model4 = df_model3.copy()
df_model4['parch'][df_model4['sex'] == 5] *= 3
df_model4

/var/folders/yk/v4q7xgf968jgjzrckxn2cqx40000gn/T/ipykernel_1500/684199604.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model4['parch'][df_model4['sex'] == 5] *= 3


,pclass,sex,age,sibsp,parch,survived
0,1,5,29.0000,0,0,1
1,1,0,0.9167,1,2,1
2,1,5,2.0000,1,6,0
3,1,0,30.0000,1,2,0
4,1,5,25.0000,1,6,0
...,...,...,...,...,...,...
1259,9,0,45.5000,0,0,0
1262,9,5,14.5000,1,0,0
1264,9,0,26.5000,0,0,0
1265,9,0,27.0000,0,0,0


In [267]:
# converting types
A = df_model4.to_numpy()    # .values gets the numpy array
A = A.astype('float64')  # so many types:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A)

[[ 1.      5.     29.      0.      0.      1.    ]
 [ 1.      0.      0.9167  1.      2.      1.    ]
 [ 1.      5.      2.      1.      6.      0.    ]
 ...
 [ 9.      0.     26.5     0.      0.      0.    ]
 [ 9.      0.     27.      0.      0.      0.    ]
 [ 9.      0.     29.      0.      0.      0.    ]]


In [268]:
NUM_ROWS, NUM_COLS = A.shape
print(f"\nThe dataset has {NUM_ROWS} rows and {NUM_COLS} cols")


The dataset has 1004 rows and 6 cols


In [269]:
# let's use all of our variables, to reinforce that we have
# (1) names...
# (2) access and control...

# choose a row index, n:
n = 42
print(f"survivor #{n} is {A[n]}")
S_INDEX = { 0 : 'female',  5 :'male'} 

for i in range(len(COLUMNS)):
    colname = COLUMNS[i]
    value = A[n][i]
    print(f"  Its {colname} is {value}")

# for sex
se_index = COL_INDEX['survived']
se_num = int(round(A[n][se_index]))
outcome = OUTCOME[se_num]
print(f" They {outcome}")

# for survived
sur_index = COL_INDEX['sex']
sur_num = int(round(A[n][sur_index]))
sex = S_INDEX[sur_num]
print(f" They are {sex} ")

survivor #42 is [ 1.  5. 53.  0.  0.  1.]
  Its pclass is 1.0
  Its sex is 5.0
  Its age is 53.0
  Its sibsp is 0.0
  Its parch is 0.0
  Its survived is 1.0
 They survived
 They are male 


In [270]:
print("+++ Start of data definitions +++\n")

#
# we could do this at the data-frame level, too!
#

X_all = A[:,0:5]  # X (features) ... is all rows, columns 0, 1, 2, 3
y_all = A[:,5]    # y (labels) ... is all rows, column 4 only

print(f"y_all (just the labels/species)   are \n {y_all}")
print(f"X_all (just the features) are \n {X_all}")

+++ Start of data definitions +++

y_all (just the labels/species)   are 
 [1. 1. 0. ... 0. 0. 0.]
X_all (just the features) are 
 [[ 1.      5.     29.      0.      0.    ]
 [ 1.      0.      0.9167  1.      2.    ]
 [ 1.      5.      2.      1.      6.    ]
 ...
 [ 9.      0.     26.5     0.      0.    ]
 [ 9.      0.     27.      0.      0.    ]
 [ 9.      0.     29.      0.      0.    ]]


In [271]:
#
# we scramble the data, to remove (potential) dependence on its ordering: 
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_labeled = X_all[indices]              # we apply the _same_ permutation to each!
y_labeled = y_all[indices]              # again...
print(f"The scrambled labels/species are \n {y_labeled}")
print(f"The corresponding data rows are \n {X_labeled}")

The scrambled labels/species are 
 [0. 0. 1. ... 0. 1. 0.]
The corresponding data rows are 
 [[ 9.  0. 20.  0.  0.]
 [ 9.  0. 18.  2.  2.]
 [ 1.  0. 40.  0.  0.]
 ...
 [ 9.  0. 28.  0.  0.]
 [ 9.  5. 27.  0.  3.]
 [ 9.  0. 26.  0.  0.]]


In [272]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)})")
print(f"y_test: {y_test}")
print(f"X_test (a few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)})")
print(f"y_train: {y_train}")
print(f"X_train (a few rows): {X_train[0:5,:]}")  # 5 rows

training with 803 rows;  testing with 201 rows

Held-out data... (testing data: 201)
y_test: [0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1.
 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1.
 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0.
 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0.
 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0.
 1. 0. 0. 1. 1. 0. 0. 1. 0.]
X_test (a few rows): [[ 9.  0. 22.  0.  0.]
 [ 9.  0. 19.  0.  0.]
 [ 9.  0. 39.  0.  2.]
 [ 9.  5. 35.  1.  3.]
 [ 4.  0. 28.  0.  1.]]

Data used for modeling... (training data: 803)
y_train: [0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1.
 1. 0. 0. 0. 0. 0. 0. 0. 1.

In [273]:
   #
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)})")
print(f"y_test: {y_test}")
print(f"X_test (a few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)})")
print(f"y_train: {y_train}")
print(f"X_train (a few rows): {X_train[0:5,:]}")  # 5 rows

training with 803 rows;  testing with 201 rows

Held-out data... (testing data: 201)
y_test: [0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1.
 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1.
 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0.
 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0.
 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0.
 1. 0. 0. 1. 1. 0. 0. 1. 0.]
X_test (a few rows): [[ 9.  0. 22.  0.  0.]
 [ 9.  0. 19.  0.  0.]
 [ 9.  0. 39.  0.  2.]
 [ 9.  5. 35.  1.  3.]
 [ 4.  0. 28.  0.  1.]]

Data used for modeling... (training data: 803)
y_train: [0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1.
 1. 0. 0. 0. 0. 0. 0. 0. 1.

In [274]:
#
# to do this, we use "cross validation"
#

from sklearn.model_selection import cross_val_score

#
# cross-validation splits the training set into two pieces:
#   + model-building and model-validation. We'll use "build" and "validate"
#
best_k = 84 # Not correct!
best_accuracy = 0.0  # also not correct...

# Note that we are cross-validating using only our TEST data!
for k in range(1,85):
    knn_cv_model = KNeighborsClassifier(n_neighbors=k)   # build knn_model for every k!
    cv_scores = cross_val_score( knn_cv_model, X_train, y_train, cv=5 )  # cv=5 means 80/20
    # print(cv_scores)  # just to see the five scores... 
    average_cv_accuracy = cv_scores.mean()  # mean() is numpy's built-in average function 
    print(f"k: {k:2d}  cv accuracy: {average_cv_accuracy:7.4f}")

    
# assign best value of k to best_k
    if average_cv_accuracy > best_accuracy:
        best_k = k    # at the moment this is incorrect   TO DO for hw4pr1: fix this...
        best_accuracy = average_cv_accuracy
# you'll need to use the loop above to find and remember the real best_k

print(f"best_k = {best_k}   yields the highest average cv accuracy.")  # print the best one


k:  1  cv accuracy:  0.7671
k:  2  cv accuracy:  0.7559
k:  3  cv accuracy:  0.8007
k:  4  cv accuracy:  0.7808
k:  5  cv accuracy:  0.7969
k:  6  cv accuracy:  0.7895
k:  7  cv accuracy:  0.8044
k:  8  cv accuracy:  0.7982
k:  9  cv accuracy:  0.7907
k: 10  cv accuracy:  0.8019
k: 11  cv accuracy:  0.8106
k: 12  cv accuracy:  0.7957
k: 13  cv accuracy:  0.7882
k: 14  cv accuracy:  0.7907
k: 15  cv accuracy:  0.7907
k: 16  cv accuracy:  0.7833
k: 17  cv accuracy:  0.7857
k: 18  cv accuracy:  0.7845
k: 19  cv accuracy:  0.7833
k: 20  cv accuracy:  0.7795
k: 21  cv accuracy:  0.7808
k: 22  cv accuracy:  0.7820
k: 23  cv accuracy:  0.7795
k: 24  cv accuracy:  0.7808
k: 25  cv accuracy:  0.7845
k: 26  cv accuracy:  0.7858
k: 27  cv accuracy:  0.7883
k: 28  cv accuracy:  0.7795
k: 29  cv accuracy:  0.7808
k: 30  cv accuracy:  0.7783
k: 31  cv accuracy:  0.7746
k: 32  cv accuracy:  0.7808
k: 33  cv accuracy:  0.7795
k: 34  cv accuracy:  0.7808
k: 35  cv accuracy:  0.7796
k: 36  cv accuracy: 

In [275]:
#
# +++ This is the "Model-building and Model-training Cell"
#       
# Create a kNN model and train it! 
#
from sklearn.neighbors import KNeighborsClassifier

k = best_k  # we don't know what k to use, so we guess!  (this will _not_ be a good value)
knn_model = KNeighborsClassifier(n_neighbors=k)       # here, k is the "k" in kNN

# we train the model (it's one line!)
knn_model.fit(X_train, y_train)                              # yay!  trained!
print("Created and trained a knn classifier with k =", k)  

Created and trained a knn classifier with k = 11


In [276]:
#
# +++ This cell will "Model-testing Cell"
#
# Now, let's see how well our model does on our "held-out data" (the testing data)
#

# We run our test set:

# the function knn_model.predict is the instantiation of our model
# it's what runs the k-nearest-neighbors algorithm:
predicted_labels = knn_model.predict(X_test)   
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual  labels  :", actual_labels)

# And, some overall results
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"\nResults on test set:  {num_correct} correct out of {total} total.")

Predicted labels: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1.
 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1.
 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0.
 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0.
 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1.
 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 1. 0.]
Actual  labels  : [0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1.
 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1.
 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0.
 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1

In [277]:
def compare_labels(predicted_labels, actual_labels):
    """ a more neatly formatted comparison """
    NUM_LABELS = len(predicted_labels)
    num_correct = 0
    
    for i in range(NUM_LABELS):
        p = int(round(predicted_labels[i]))         # round protects from fp error 
        a = int(round(actual_labels[i]))
        result = "incorrect"
        if p == a:  # if they match,
            result = ""       # no longer incorrect
            num_correct += 1  # and we count a match!

        print(f"row {i:>3d} : {OUTCOME[p]:>12s} {OUTCOME[a]:<12s}   {result}")   

    print()
    print("Correct:", num_correct, "out of", NUM_LABELS)
    return num_correct

# let's try it out!
compare_labels(predicted_labels,actual_labels)

row   0 :     perished perished       
row   1 :     perished perished       
row   2 :     perished perished       
row   3 :     perished survived       incorrect
row   4 :     perished perished       
row   5 :     perished survived       incorrect
row   6 :     perished survived       incorrect
row   7 :     survived perished       incorrect
row   8 :     perished perished       
row   9 :     perished perished       
row  10 :     perished perished       
row  11 :     survived survived       
row  12 :     perished survived       incorrect
row  13 :     survived survived       
row  14 :     perished perished       
row  15 :     perished perished       
row  16 :     perished perished       
row  17 :     perished perished       
row  18 :     perished survived       incorrect
row  19 :     survived survived       
row  20 :     perished perished       
row  21 :     perished perished       
row  22 :     survived survived       
row  23 :     survived survived       
row  24 : 

153

In [278]:
# Ok!  We have tuned knn to use the "best" value of k...
#
# And, we should now use ALL available data to train our final predictive model:
#

knn_model_final = KNeighborsClassifier(n_neighbors=best_k)   # here, we use the best_k
knn_model_final.fit(X_all, y_all)                              # here we use ALL the data!
print(f"Created + trained a 'final' knn classifier, with a (best) k of {best_k}") 

Created + trained a 'final' knn classifier, with a (best) k of 11


In [242]:
#
# final predictive model (k-nearest-neighbor), with tuned k + ALL data incorporated
#

def predictive_model( Features, Model ):
    """ input: a list of five features 
            [ pclass' (ticket class, 1-3), 
            'sex', (male (0), female (5))
            'age',
            'sibsp' (# of siblings or spouses)
            'parch', (# of children or parents)]
        output: the predicted outcome of individuals, from
                  survived (1), perished (0)
    """
    # put features into a list!
    our_features = np.asarray([Features])                 # extra brackets needed

    # The model's prediction!
    predicted_outcome = Model.predict(our_features)
    
    # a bit awkward
    predicted_outcome = int(round(predicted_outcome[0]))  # unpack one element
    return predicted_outcome

    
#
# Try it!
# 
Features = eval(input("Enter new Features: "))

# Features = [6.7,3.3,5.7,0.1]  # [5.8,2.7,4.1,1.0] [4.6,3.6,3.0,2.2] [6.7,3.3,5.7,2.1]

predicted_outcome = predictive_model( Features, knn_model_final )  # pass in the model, too!
name = OUTCOME[predicted_outcome]
print(f"I predict {name} from the features {Features}")

# LoF = [
# [4.8, 3.1, 1.6, 0.2 ],
# [5.7, 2.9, 4.2, 1.3 ],
# [5.8, 2.7, 5.1, 1.9 ],
# [5.2, 4.1, 1.5, 0.1 ],
# [5.4, 3.4, 1.5, 0.4 ],
# [5.1, 2.5, 3.0, 1.1 ],
# [6.2, 2.9, 4.3, 1.3 ],
# [6.3, 3.3, 6.0, 2.5 ],
# [5.7, 2.8, 4.1, 1.3 ],
#   ]

# run on each one:
# for Features in LoF:
    # predicted_outcome = predictive_model( Features, knn_model_final )  # pass in the model, too!
    # name = OUTCOME[predicted_outcome]
    # print(f"I predict {name} from the features {Features}")

I predict survived from the features (1, 5, 20, 0, 2)


### Thoughts on Titanic
- best cross validation accuracy: 0.8106 when k = 11
- which feature is most important
    - Weighting the class was most important. Before with no weighting, the accuracy was 128/200 correct predictions. After weighting the 3rd and 2nd classes progressively more heavily, the accuracy was 151/200. Weighting sex (parents with children) also helped with the accuracy.
